<a href="https://colab.research.google.com/github/dwiawen/smartcity/blob/main/data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweet-preprocessor
!pip install textblob
!pip install sastrawi
!pip install emoji
!pip install PySastrawi
!pip install stopwords

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=4d4844fa4b26a97e20ead18c711ff65f8482c2050ea99a820cb910ae5670a211
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 17.3 MB/s eta 0:00:00


In [ ]:
from nltk.tree.immutable import ImmutableParentedTree
import os
import pandas as pd
import re
import string
from textblob import TextBlob
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime
from datetime import timedelta
import numpy as np
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
# Read data
import pandas as pd
def load_data():
  data = pd.read_csv('/content/drive/MyDrive/SENTIMENT ANALYSIS RESEARCH/transportasi.csv')
  return data

In [ ]:
tweet_df = load_data()

In [ ]:
# Buka tweet yang berisi username, date, dan tweet dari crawling data twitter
tweet_df = pd.DataFrame(tweet_df[['Username', 'Tweet', 'Date']])
tweet_df.head()

,Username,Tweet,Date
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00


In [ ]:
# Remove mention username dan retweet
def remove_pattern(Tweet, pattern_regex):
    r = re.findall(pattern_regex, Tweet)
    for i in r:
        Tweet = re.sub(re.escape(i), '', Tweet)
    return Tweet

# Define the regular expression pattern
pattern = r'RT|@\w+|"'

# Apply the remove_pattern function to clean the tweets
tweet_df['Clean_Tweet'] = np.vectorize(remove_pattern)(tweet_df['Tweet'], pattern)

# Display the updated DataFrame
tweet_df.head(5)

,Username,Tweet,Date,Clean_Tweet
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda..."
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny..."
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...


In [ ]:
# Define the remove_symbols function
def remove_symbols(text):
    pattern = r'[^\w\s]'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# Apply the remove_symbols function to clean the tweets
tweet_df['Remove_Http'] = tweet_df['Clean_Tweet'].apply(lambda x: remove_symbols(x))

# Display the updated DataFrame
tweet_df.head()

,Username,Tweet,Date,Clean_Tweet,Remove_Http
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...


In [ ]:
# Remove function
def remove(tweet):
    # Remove stock market ticker like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # Remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # Remove Hashtags
    # Only removing the hash # sign from the word
    tweet = re.sub('#', '', tweet)

    # Remove numbers
    tweet = re.sub('[0-9]+', '', tweet)

    return tweet

# Apply the remove function to clean the tweets
tweet_df['Remove_Hashtag'] = tweet_df['Remove_Http'].apply(lambda x: remove(x))

# Display the updated DataFrame
tweet_df.head()

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...


In [ ]:
# Remove tweet kosong
tweet_df = tweet_df[tweet_df["Tweet"] != ""]
tweet_df.head()

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...


In [ ]:
# Remove data duplikat
tweet_df.drop_duplicates(subset="Remove_Hashtag", keep='first', inplace=True)
tweet_df.head()

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...


In [ ]:
from tkinter.constants import X
# Import Stop Watch

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

from Sastrawi.Dictionary.ArrayDictionary import ArrayDictionary
from Sastrawi.StopWordRemover.StopWordRemover import StopWordRemover

stop_factory = StopWordRemoverFactory().get_stop_words()

more_stopwords = [
            'a','ada','adalah','adanya','adapun','agak','agaknya','agar','akan','akankah','akhir',
            'akhiri','akhirnya','aku','akulah','amat','amatlah','anda','andalah','antar','antara',
            'antaranya','apa','apaan','apabila','apakah','apalagi','apatah','arti','artinya','asal',
            'asalkan','atas','atau','ataukah','ataupun','awal','awalnya','b','bagai','bagaikan',
            'bagaimana','bagaimanakah','bagaimanapun','bagainamakah','bagi','bagian','bahkan','bahwa',
            'bahwasannya','bahwasanya','baik','baiklah','bakal','bakalan','balik','banyak','bapak',
            'baru','bawah','beberapa','begini','beginian','beginikah','beginilah','begitu','begitukah',
            'begitulah','begitupun','bekerja','belakang','belakangan','belum','belumlah','benar',
            'benarkah','benarlah','berada','berakhir','berakhirlah','berakhirnya','berapa','berapakah',
            'berapalah','berapapun','berarti','berawal','berbagai','berdatangan','beri','berikan',
            'berikut','berikutnya','berjumlah','berkali-kali','berkata','berkehendak','berkeinginan',
            'berkenaan','berlainan','berlalu','berlangsung','berlebihan','bermacam','bermacam-macam',
            'bermaksud','bermula','bersama','bersama-sama','bersiap','bersiap-siap','bertanya',
            'bertanya-tanya','berturut','berturut-turut','bertutur','berujar','berupa','besar',
            'betul','betulkah','biasa','biasanya','bila','bilakah','bisa','bisakah','boleh','bolehkah',
            'bolehlah','buat','bukan','bukankah','bukanlah','bukannya','bulan','bung','c','cara',
            'caranya','cukup','cukupkah','cukuplah','cuma','d','dahulu','dalam','dan','dapat','dari',
            'daripada','datang','dekat','demi','demikian','demikianlah','dengan','depan','di','dia',
            'diakhiri','diakhirinya','dialah','diantara','diantaranya','diberi','diberikan','diberikannya',
            'dibuat','dibuatnya','didapat','didatangkan','digunakan','diibaratkan','diibaratkannya',
            'diingat','diingatkan','diinginkan','dijawab','dijelaskan','dijelaskannya','dikarenakan',
            'dikatakan','dikatakannya','dikerjakan','diketahui','diketahuinya','dikira','dilakukan',
            'dilalui','dilihat','dimaksud','dimaksudkan','dimaksudkannya','dimaksudnya','diminta',
            'dimintai','dimisalkan','dimulai','dimulailah','dimulainya','dimungkinkan','dini','dipastikan',
            'diperbuat','diperbuatnya','dipergunakan','diperkirakan','diperlihatkan','diperlukan',
            'diperlukannya','dipersoalkan','dipertanyakan','dipunyai','diri','dirinya','disampaikan',
            'disebut','disebutkan','disebutkannya','disini','disinilah','ditambahkan','ditandaskan',
            'ditanya','ditanyai','ditanyakan','ditegaskan','ditujukan','ditunjuk','ditunjuki','ditunjukkan',
            'ditunjukkannya','ditunjuknya','dituturkan','dituturkannya','diucapkan','diucapkannya',
            'diungkapkan','dong','dua','dulu','e','empat','enak','enggak','enggaknya','entah','entahlah',
            'f','g','guna','gunakan','h','hadap','hai','hal','halo','hallo','hampir','hanya','hanyalah',
            'hari','harus','haruslah','harusnya','helo','hello','hendak','hendaklah','hendaknya','hingga',
            'i','ia','ialah','ibarat','ibaratkan','ibaratnya','ibu','ikut','ingat','ingat-ingat','ingin',
            'inginkah','inginkan','ini','inikah','inilah','itu','itukah','itulah','j','jadi','jadilah',
            'jadinya','jangan','jangankan','janganlah','jauh','jawab','jawaban','jawabnya','jelas',
            'jelaskan','jelaslah','jelasnya','jika','jikalau','juga','jumlah','jumlahnya','justru',
            'k','kadar','kala','kalau','kalaulah','kalaupun','kali','kalian','kami','kamilah','kamu',
            'kamulah','kan','kapan','kapankah','kapanpun','karena','karenanya','kasus','kata','katakan',
            'katakanlah','katanya','ke','keadaan','kebetulan','kecil','kedua','keduanya','keinginan',
            'kelamaan','kelihatan','kelihatannya','kelima','keluar','kembali','kemudian','kemungkinan',
            'kemungkinannya','kena','kenapa','kepada','kepadanya','kerja','kesampaian','keseluruhan',
            'keseluruhannya','keterlaluan','ketika','khusus','khususnya','kini','kinilah','kira',
            'kira-kira','kiranya','kita','kitalah','kok','kurang','l','lagi','lagian','lah','lain',
            'lainnya','laku','lalu','lama','lamanya','langsung','lanjut','lanjutnya','lebih','lewat',
            'lihat','lima','luar','m','macam','maka','makanya','makin','maksud','malah','malahan',
            'mampu','mampukah','mana','manakala','manalagi','masa','masalah','masalahnya','masih',
            'masihkah','masing','masing-masing','masuk','mata','mau','maupun','melainkan','melakukan',
            'melalui','melihat','melihatnya','memang','memastikan','memberi','memberikan','membuat',
            'memerlukan','memihak','meminta','memintakan','memisalkan','memperbuat','mempergunakan',
            'memperkirakan','memperlihatkan','mempersiapkan','mempersoalkan','mempertanyakan','mempunyai',
            'memulai','memungkinkan','menaiki','menambahkan','menandaskan','menanti','menanti-nanti',
            'menantikan','menanya','menanyai','menanyakan','mendapat','mendapatkan','mendatang','mendatangi',
            'mendatangkan','menegaskan','mengakhiri','mengapa','mengatakan','mengatakannya','mengenai',
            'mengerjakan','mengetahui','menggunakan','menghendaki','mengibaratkan','mengibaratkannya',
            'mengingat','mengingatkan','menginginkan','mengira','mengucapkan','mengucapkannya','mengungkapkan',
            'menjadi','menjawab','menjelaskan','menuju','menunjuk','menunjuki','menunjukkan','menunjuknya',
            'menurut','menuturkan','menyampaikan','menyangkut','menyatakan','menyebutkan','menyeluruh',
            'menyiapkan','merasa','mereka','merekalah','merupakan','meski','meskipun','meyakini','meyakinkan',
            'minta','mirip','misal','misalkan','misalnya','mohon','mula','mulai','mulailah','mulanya','mungkin',
            'mungkinkah','n','nah','naik','namun','nanti','nantinya','nya','nyaris','nyata','nyatanya',
            'o','oleh','olehnya','orang','p','pada','padahal','padanya','pak','paling','panjang','pantas',
            'para','pasti','pastilah','penting','pentingnya','per','percuma','perlu','perlukah','perlunya',
            'pernah','persoalan','pertama','pertama-tama','pertanyaan','pertanyakan','pihak','pihaknya',
            'pukul','pula','pun','punya','q','r','rasa','rasanya','rupa','rupanya','s','saat','saatnya','saja',
            'sajalah','salam','saling','sama','sama-sama','sambil','sampai','sampai-sampai','sampaikan','sana',
            'sangat','sangatlah','sangkut','satu','saya','sayalah','se','sebab','sebabnya','sebagai',
            'sebagaimana','sebagainya','sebagian','sebaik','sebaik-baiknya','sebaiknya','sebaliknya',
            'sebanyak','sebegini','sebegitu','sebelum','sebelumnya','sebenarnya','seberapa','sebesar',
            'sebetulnya','sebisanya','sebuah','sebut','sebutlah','sebutnya','secara','secukupnya','sedang',
            'sedangkan','sedemikian','sedikit','sedikitnya','seenaknya','segala','segalanya','segera',
            'seharusnya','sehingga','seingat','sejak','sejauh','sejenak','sejumlah','sekadar','sekadarnya',
            'sekali','sekali-kali','sekalian','sekaligus','sekalipun','sekarang','sekaranglah','sekecil',
            'seketika','sekiranya','sekitar','sekitarnya','sekurang-kurangnya','sekurangnya','sela','selain',
            'selaku','selalu','selama','selama-lamanya','selamanya','selanjutnya','seluruh','seluruhnya',
            'semacam','semakin','semampu','semampunya','semasa','semasih','semata','semata-mata','semaunya',
            'sementara','semisal','semisalnya','sempat','semua','semuanya','semula','sendiri','sendirian',
            'sendirinya','seolah','seolah-olah','seorang','sepanjang','sepantasnya','sepantasnyalah',
            'seperlunya','seperti','sepertinya','sepihak','sering','seringnya','serta','serupa','sesaat',
            'sesama','sesampai','sesegera','sesekali','seseorang','sesuatu','sesuatunya','sesudah',
            'sesudahnya','setelah','setempat','setengah','seterusnya','setiap','setiba','setibanya',
            'setidak-tidaknya','setidaknya','setinggi','seusai','sewaktu','siap','siapa','siapakah',
            'siapapun','sini','sinilah','soal','soalnya','suatu','sudah','sudahkah','sudahlah','supaya',
            't','tadi','tadinya','tahu','tak','tambah','tambahnya','tampak','tampaknya','tandas','tandasnya',
            'tanpa','tanya','tanyakan','tanyanya','tapi','tegas','tegasnya','telah','tempat','tentang','tentu',
            'tentulah','tentunya','tepat','terakhir','terasa','terbanyak','terdahulu','terdapat','terdiri',
            'terhadap','terhadapnya','teringat','teringat-ingat','terjadi','terjadilah','terjadinya','terkira',
            'terlalu','terlebih','terlihat','termasuk','ternyata','tersampaikan','tersebut','tersebutlah',
            'tertentu','tertuju','terus','terutama','tetap','tetapi','tiap','tiba','tiba-tiba','tidak',
            'tidakkah','tidaklah','tiga','toh','tuju','tunjuk','turut','tutur','tuturnya','u','ucap','ucapnya',
            'ujar','ujarnya','umumnya','ungkap','ungkapnya','untuk','usah','usai','v','w','waduh','wah','wahai',
            'waktunya','walau','walaupun','wong','x','y','ya','yaitu','yakin','yakni','yang','z', 'udah', 'mnrt',
            'sy', 'sdh', 'sgt', 'jk', 'sm', 'iya', 'jg', 'sm']

data = stop_factory + more_stopwords

dictionary = ArrayDictionary(data)
str = StopWordRemover(dictionary)

print(data)

# create stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# tokenize
from nltk.tokenize import TweetTokenizer

#happy emoticon
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

# All emoticons (happy+sad)
emoticons = emoticons_happy.union(emoticons_sad)

from nltk.tokenize import TweetTokenizer
import string

def clean_tweets(tweet):
    # Tokenize tweet
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweet_clean = []
    for word in tweet_tokens:
        if (word not in data and #remove stopword
            word not in emoticons and  #remove emoticons
            word not in string.punctuation):  #remove punctuation
            #tweets_clean.append(stem_word)
            stem_word = stemmer.stem(word)
            tweet_clean.append(stem_word)

    return tweet_clean

tweet_df['Tweet_Text'] = tweet_df['Remove_Hashtag'].apply(lambda x: clean_tweets(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['a', 'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'arti', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'b', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagainamakah', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasannya', 'bahwasanya', 'baik', 'baiklah', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumla

In [ ]:
tweet_df.head() # Tokenization

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag,Tweet_Text
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,"[kembang, sistem, transportasi, umum, massal, ..."
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...,"[tiket, makan, minum, transportasi, akomodasi,..."
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,"[transportasi, publik, jakarta, bagus, sedia, ..."
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...,"[lokasi, konser, butuh, kendara, gbk, opsi, ba..."
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,"[dukung, kurang, emisi, elektrifikasi, kendara..."


In [ ]:
# Remove Punct
def remove_punct(text):
  text = " ".join([char for char in text if char not in string.punctuation])
  return text
tweet_df['Tweet_Text'] = tweet_df['Tweet_Text'].apply(lambda x: remove_punct(x))


In [ ]:
tweet_df.head()

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag,Tweet_Text
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,kembang sistem transportasi umum massal integr...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...,tiket makan minum transportasi akomodasi akomo...
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,transportasi publik jakarta bagus sedia bnyk
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...,lokasi konser butuh kendara gbk opsi bawa kend...
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,dukung kurang emisi elektrifikasi kendara moto...


In [ ]:
# Remove Tweet Kosong
tweet_df = tweet_df[tweet_df['Tweet_Text']!=""]
tweet_df.head()

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag,Tweet_Text
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,kembang sistem transportasi umum massal integr...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...,tiket makan minum transportasi akomodasi akomo...
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,transportasi publik jakarta bagus sedia bnyk
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...,lokasi konser butuh kendara gbk opsi bawa kend...
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,dukung kurang emisi elektrifikasi kendara moto...


In [ ]:
# Reset Index
tweet_df = tweet_df.reset_index(drop=True)
tweet_df.head()

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag,Tweet_Text
0,dawiguna,"""Pengembangan sistem transportasi umum massal ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,kembang sistem transportasi umum massal integr...
1,kapankayaid,"Tiket udah, makan minum udah, transportasi uda...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...,tiket makan minum transportasi akomodasi akomo...
2,TyarEksposa,@kompascom Transportasi publik di jakarta mnrt...,2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,transportasi publik jakarta bagus sedia bnyk
3,kapankayaid,"Lanjut, untuk menuju ke lokasi konser, tentuny...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...,lokasi konser butuh kendara gbk opsi bawa kend...
4,kompascom,"""Saya tidak mendukung pengurangan emisi lewat ...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,dukung kurang emisi elektrifikasi kendara moto...


In [ ]:
columns_to_drop = [0, 1, 2, 3, 4, 5]

# Check if columns exist before dropping
existing_columns = tweet_df.columns
columns_to_drop = [col_idx for col_idx in columns_to_drop if col_idx < len(existing_columns)]

if columns_to_drop:
    tweet_df.drop(existing_columns[columns_to_drop], axis=1, inplace=True)

tweet_df.head()

,Tweet_Text
0,kembang sistem transportasi umum massal integr...
1,tiket makan minum transportasi akomodasi akomo...
2,transportasi publik jakarta bagus sedia bnyk
3,lokasi konser butuh kendara gbk opsi bawa kend...
4,dukung kurang emisi elektrifikasi kendara moto...


In [ ]:
# Simpan Data Bersih
tweet_df.to_csv('transportasibersih.csv', index=False)

In [ ]:
# Install Google Translate
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=add6a975faa23cd59aca1c3df1ee7db726188bf077146288636e15b685c70bf1
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans


In [ ]:
# Import Library yang diperlukan
import pandas as pd
import googletrans
from googletrans import Translator

In [ ]:
!pip install translate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Membaca dan menyiapkan data csv sebagai dataframe
df = pd.read_csv('/content/bersih_transportasi.csv')
df.head(5)

,Username,Tweet,Date,Clean_Tweet,Remove_Http,Remove_Hashtag
0,dawiguna,"['kembang', 'sistem', 'transportasi', 'umum', ...",2023-06-12 23:59:10+00:00,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...,Pengembangan sistem transportasi umum massal t...
1,kapankayaid,"['tiket', 'udah', 'makan', 'minum', 'udah', 't...",2023-06-12 23:57:46+00:00,"Tiket udah, makan minum udah, transportasi uda...",Tiket udah makan minum udah transportasi udah ...,Tiket udah makan minum udah transportasi udah ...
2,TyarEksposa,"['transportasi', 'publik', 'jakarta', 'mnrt', ...",2023-06-12 23:54:13+00:00,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...,Transportasi publik di jakarta mnrt sy Sdh sg...
3,kapankayaid,"['lokasi', 'konser', 'butuh', 'kendara', 'gbk'...",2023-06-12 23:53:54+00:00,"Lanjut, untuk menuju ke lokasi konser, tentuny...",Lanjut untuk menuju ke lokasi konser tentunya ...,Lanjut untuk menuju ke lokasi konser tentunya ...
4,kompascom,"['dukung', 'kurang', 'emisi', 'elektrifikasi',...",2023-06-12 23:48:20+00:00,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...,Saya tidak mendukung pengurangan emisi lewat e...


In [ ]:
from translate import Translator

translator = Translator(from_lang='id', to_lang='en')
translations = {}

for column in df.columns:
    unique_elements = df[column].unique()
    for element in unique_elements:
        translations[element] = translator.translate(element)

translations

{'kembang sistem transportasi umum massal integrasi imbang laut udara darat komitmen kuasa sumber daya alam kurang subsidi bbm triliun turun harga energi persen kurang utang negara tahap rasio utang pdb kecil catat hutang utk oleh aset pisah hitung hutang produktif negara bangun irigasi bendung rehabilitasi irigasi rusak juta ha tani bendung tahun tuh capek copasnya sisa google': 'MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  15 HOURS 19 MINUTES 49 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE', 'tiket makan minum transportasi akomodasi akomodasi gak laku kota capek pulang gak dapet kendara umum singapore nambah': 'MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRANSLATIONS FOR TODAY. NEXT AVAILABLE IN  15 HOURS 19 MINUTES 48 SECONDS VISIT HTTPS://MYMEMORY.TRANSLATED.NET/DOC/USAGELIMITS.PHP TO TRANSLATE MORE', 'transportasi publik jakarta bagus sedia bnyk': 'MYMEMORY WARNING: YOU USED ALL AVAILABLE FREE TRA

In [ ]:
df.replace(translations, inplace = True)
df.head(10)

,Tweet_Text
0,kembang sistem transportasi umum massal integr...
1,tiket makan minum transportasi akomodasi akomo...
2,transportasi publik jakarta bagus sedia bnyk
3,lokasi konser butuh kendara gbk opsi bawa kend...
4,dukung kurang emisi elektrifikasi kendara moto...
5,mikir gin pas diskusi ortu jt bayar up sdp bpp...
6,tim pake masker umum kayak transportasi umum j...
7,transpoasi kelas kereta api pesawat terbang yg...
8,pagi semesta alat transportasi lucu kereta api...
9,ngapunten kl utk kondisi rejekinya niat ingsun...


In [ ]:
translator = Translator()
translations = {}
for column in df.columns:
    # unique elements of the column
    unique_elements = df[column].unique()
    for element in unique_elements:
        # add translation to the dictionary
        translations[element] = translator.translate(element)

print(translations)

TypeError: ignored